In [1]:
import random as ra
import pandas as pd
import string
import hungarian_algorithm.algorithm as ha
import itertools as it

**Import the table of visual similarities between lower-case letters.** For this step to work, you will need to download the file `13428_2012_271_MOESM1_ESM.xlsx` which is available in the "Electronic supplementary material" for the research paper
[A letter visual-similarity matrix for Latin-based alphabets](https://link.springer.com/article/10.3758/s13428-012-0271-4).

As described in the paper, two forms of letter 'a' are included in the table; we average the similarities for the two forms of 'a'.

In [2]:
def read_similarity_table():
    
    input_filename = "13428_2012_271_MOESM1_ESM.xlsx"
    
    df = pd.read_excel(input_filename, sheet_name="List-Lower", usecols="C:E")
    df = df.loc[df['Letter1'].isin(list(string.ascii_lowercase))]
    df = df.loc[df['Letter2'].isin(list(string.ascii_lowercase))]
    df = df.loc[df['Letter1'] != df['Letter2']]

    df = df.groupby(['Letter1', 'Letter2']).agg(similarity = ('Value', 'mean')).reset_index()    

    df['Key'] = df.apply(lambda row: (row['Letter1'], row['Letter2']), axis='columns')

    assert df.shape[0] == 26 * 25
    
    return df


In [3]:
sim_df = read_similarity_table()

**Which letter pairs were judged the most similar? Which were judged the least similar?**

In [4]:
sim_df.loc[sim_df['Letter1'] < sim_df['Letter2']].sort_values('similarity', ascending=False).head(5)

,Letter1,Letter2,similarity,Key
210,i,l,6.133333,"(i, l)"
27,b,d,5.600000,"(b, d)"
390,p,q,5.566667,"(p, q)"
187,h,n,5.533333,"(h, n)"
548,v,y,5.333333,"(v, y)"


In [5]:
sim_df.loc[sim_df['Letter1'] < sim_df['Letter2']].sort_values('similarity').head(5)

,Letter1,Letter2,similarity,Key
238,j,o,1.000000,"(j, o)"
496,t,w,1.000000,"(t, w)"
421,q,w,1.033333,"(q, w)"
446,r,w,1.033333,"(r, w)"
263,k,o,1.033333,"(k, o)"


In [6]:
sim_dict = sim_df.set_index('Key')['similarity'].to_dict()

We decree that every letter is highly similar to itself.

In [7]:
self_similarity = sim_df['similarity'].max() + 1

In [8]:
for letter in string.ascii_lowercase:
    sim_dict[(letter, letter)] = self_similarity

**Function to measure the total similarity of a rearrangement of a word.**

In [9]:
def rearr_similarity(word, rearr):
    return sum([sim_dict[(x, y)] for x, y in zip(word, rearr)])

In [10]:
rearr_similarity("boat", "baot")

22.4

In [11]:
rearr_similarity("boat", "taob")

11.399999999999999

In [12]:
rearr_similarity("boat", "atob")

9.666666666666666

**Function to build the dictionary representation of the assignment problem needed by `hungarian-algorithm` library.**

In [13]:
def mk_assignment_problem(word):

    def options_for_letter(letter):
        return {'_%d' % i: self_similarity - sim_dict[(letter, word[i])] for i in range(len(word))}
    
    return {str(i): options_for_letter(word[i]) for i in range(len(word))}

**Function to build the assignment problem, solve it, and interpret the results into a rearranged word.**

In [14]:
def rearrange_word(word):
    matching = ha.find_matching(mk_assignment_problem(word))

    def recover_mapping(x):
        src = int(x[0][0])
        dst = int(x[0][1][1:])
        return (src, dst)
    
    permutation = list(map(recover_mapping, matching))
    permutation_dict = {src: dst for src, dst in permutation}
    
    return "".join([word[permutation_dict[i]] for i in range(len(word))])
    
    return permutation_dict

In [15]:
rearrange_word("orange")

'ronaeg'

In [16]:
rearrange_word("boat")

'atbo'

**Bit of testing:** let's enumerate all the permutations of a word and measure them, and see if we got the right answer... (Only run this on relatively short words...)

In [17]:
test_word = "questions"

In [18]:
rearr = rearrange_word(test_word)
rearr

'ssiunoteq'

In [19]:
rearr_similarity(test_word, rearr)

11.633333333333333

In [20]:
all_rearr_words = ["".join(r) for r in it.permutations(test_word)]

In [21]:
len(all_rearr_words)

362880

In [22]:
all_rearr_words_scored_df = pd.DataFrame(
    list(map(lambda w: (w, rearr_similarity(test_word, w)), all_rearr_words)),
    columns=['rearrangement', 'similarity']).drop_duplicates('rearrangement')

The most visually similar rearrangements:

In [23]:
all_rearr_words_scored_df.sort_values('similarity', ascending=False).round(2).head(5)

,rearrangement,similarity
0,questions,64.20
31112,qnestious,59.00
3030,quostiens,58.20
247830,ouestiqns,58.13
24,quesitons,57.73


The least visually similar rearragements:

In [24]:
all_rearr_words_scored_df.sort_values('similarity').round(2).head(5)

,rearrangement,similarity
160787,ssnuoeitq,11.63
159359,ssiunoteq,11.63
160667,ssnqoeitu,11.63
159239,ssiqnoteu,11.63
133407,seiunotsq,11.65


Did the Hungarian Algorithm give us the same word? (Sometimes there can be multiple options with the same score.)

In [25]:
all_rearr_words_scored_df.sort_values('similarity').head(1)['rearrangement'].item() == rearr

False

In [26]:
abs(
      rearr_similarity(test_word, all_rearr_words_scored_df.sort_values('similarity').head(1)['rearrangement'].item())
    - rearr_similarity(test_word, rearr)
)

0.0

**Put it all together and run on an input sentence.**

The word splitting etc. here is really simplistic, it only words on lower-case letters and I don't deal with punctuation properly, sorry; I concentrated on an aspect of the problem that was interesting to me!


In [27]:
def rearrange_text(text):
    return " ".join(map(rearrange_word, text.split(" ")))

In [28]:
rearrange_text("happy new year everybody")

'aphyp ewn arye revobeyyd'

😢😢 🐛**BUG**🐛 😢😢:

There's definitely some kind of bug here. When the code runs successfully, the results look right, but here are some things that seem to hang (despite some longer words running without issue):

In [29]:
#rearrange_word("arrange")

In [30]:
#rearrange_word("greetings")

I haven't got time to investigate sadly.